# Enhanced Technical Analysis Dashboard

# Enhanced Technical Analysis Dashboard

This notebook provides a comprehensive technical analysis dashboard with:
1. Interactive price charts with multiple technical indicators
2. Market data and fundamental analysis
3. Trading signal analysis
4. Comparative analysis across stocks
5. Performance metrics and KPIs

In [1]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta

# Add src directory to path
sys.path.append(os.path.abspath('..'))
from src.stock_data_loader import load_stock_data
from src.ta_analysis import TechnicalAnalyzer

# Set plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

## 1. Load and Prepare Data

In [2]:
# Load stock data
tickers = ["AAPL", "AMZN", "GOOG", "META", "MSFT", "NVDA", "TSLA"]
stock_data = load_stock_data(tickers, data_dir="../data")

# Initialize technical analyzers
analyzers = {}
for ticker in stock_data:
    print(f"\nInitializing analyzer for {ticker}...")
    df_with_date = stock_data[ticker].reset_index()
    analyzers[ticker] = TechnicalAnalyzer(df_with_date, symbol=ticker)
    
# Print data summary
print("\nData Summary:")
for ticker in analyzers:
    print(f"\n{ticker}:")
    print(f"Date Range: {analyzers[ticker].df['Date'].min()} to {analyzers[ticker].df['Date'].max()}")
    print(f"Number of trading days: {len(analyzers[ticker].df)}")

Loaded AAPL data: 10998 rows from 1980-12-12 00:00:00 to 2024-07-30 00:00:00
Loaded AMZN data: 6846 rows from 1997-05-15 00:00:00 to 2024-07-30 00:00:00
Loaded GOOG data: 5020 rows from 2004-08-19 00:00:00 to 2024-07-30 00:00:00
Loaded META data: 2926 rows from 2012-12-12 00:00:00 to 2024-07-30 00:00:00
Loaded MSFT data: 9672 rows from 1986-03-13 00:00:00 to 2024-07-30 00:00:00
Loaded NVDA data: 6421 rows from 1999-01-22 00:00:00 to 2024-07-30 00:00:00
Loaded TSLA data: 3545 rows from 2010-06-29 00:00:00 to 2024-07-30 00:00:00

Initializing analyzer for AAPL...

Initializing analyzer for AMZN...

Initializing analyzer for GOOG...

Initializing analyzer for META...

Initializing analyzer for MSFT...

Initializing analyzer for NVDA...

Initializing analyzer for TSLA...

Data Summary:

AAPL:
Date Range: 1981-09-28 00:00:00 to 2024-07-30 00:00:00
Number of trading days: 10799

AMZN:
Date Range: 1998-03-02 00:00:00 to 2024-07-30 00:00:00
Number of trading days: 6647

GOOG:
Date Range: 2005-

## 2. Interactive Dashboard

In [3]:
def create_stock_dashboard(ticker, days=100):
    """Create an interactive dashboard for a single stock."""
    analyzer = analyzers[ticker]
    df = analyzer.df.copy()
    
    # Filter for recent data
    end_date = df['Date'].max()
    start_date = end_date - pd.Timedelta(days=days)
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    # Create subplots
    fig = make_subplots(
        rows=5, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        row_heights=[0.4, 0.15, 0.15, 0.15, 0.15],
        subplot_titles=(
            f'{ticker} Price and Indicators',
            'Volume and VWAP',
            'RSI and Stochastic',
            'MACD',
            'Bollinger Bands'
        )
    )
    
    # 1. Price and Moving Averages
    fig.add_trace(
        go.Candlestick(
            x=df['Date'],
            open=df['Open'],
            high=df['High'],
            low=df['Low'],
            close=df['Close'],
            name='OHLC'
        ),
        row=1, col=1
    )
    
    # Add moving averages
    for ma, color in [('SMA_20', 'blue'), ('SMA_50', 'orange'), ('SMA_200', 'red')]:
        fig.add_trace(
            go.Scatter(
                x=df['Date'],
                y=df[ma],
                name=ma,
                line=dict(color=color)
            ),
            row=1, col=1
        )
    
    # 2. Volume and VWAP
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df['Volume'],
            name='Volume',
            marker_color='rgba(0,0,255,0.3)'
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['VWAP'],
            name='VWAP',
            line=dict(color='purple')
        ),
        row=2, col=1
    )
    
    # 3. RSI and Stochastic
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['RSI'],
            name='RSI',
            line=dict(color='blue')
        ),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['Stoch_k'],
            name='Stochastic %K',
            line=dict(color='green')
        ),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['Stoch_d'],
            name='Stochastic %D',
            line=dict(color='red')
        ),
        row=3, col=1
    )
    
    # Add RSI levels
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)
    
    # 4. MACD
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['MACD'],
            name='MACD',
            line=dict(color='blue')
        ),
        row=4, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['MACD_signal'],
            name='Signal',
            line=dict(color='red')
        ),
        row=4, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df['Date'],
            y=df['MACD_diff'],
            name='MACD Diff',
            marker_color='rgba(0,255,0,0.3)'
        ),
        row=4, col=1
    )
    
    # 5. Bollinger Bands
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['BB_high'],
            name='BB Upper',
            line=dict(color='gray', dash='dash')
        ),
        row=5, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['BB_low'],
            name='BB Lower',
            line=dict(color='gray', dash='dash')
        ),
        row=5, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['Date'],
            y=df['BB_mid'],
            name='BB Middle',
            line=dict(color='gray')
        ),
        row=5, col=1
    )
    
    # Update layout
    fig.update_layout(
        title=f'{ticker} Technical Analysis Dashboard',
        yaxis_title='Price',
        yaxis2_title='Volume',
        yaxis3_title='RSI/Stochastic',
        yaxis4_title='MACD',
        yaxis5_title='Bollinger Bands',
        xaxis_rangeslider_visible=False,
        height=1200,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

# Create dashboard for each stock
for ticker in analyzers:
    print(f"\nGenerating dashboard for {ticker}...")
    fig = create_stock_dashboard(ticker)
    fig.show()


Generating dashboard for AAPL...



Generating dashboard for AMZN...



Generating dashboard for GOOG...



Generating dashboard for META...



Generating dashboard for MSFT...



Generating dashboard for NVDA...



Generating dashboard for TSLA...


## 3. Market Data and Fundamental Analysis

In [7]:
def create_market_summary(ticker):
    """Create a market summary dashboard for a stock."""
    analyzer = analyzers[ticker]
    summary = analyzer.get_market_summary()
    
    # Create subplots for different metrics
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Technical Indicators',
            'Fundamental Metrics',
            'Performance Metrics',
            'Market Sentiment'
        )
    )
    
    # 1. Technical Indicators
    tech_data = summary['Technical']
    
    # Price and Moving Averages
    price_data = {
        'Current': tech_data['Price']['Current'],
        'SMA 20': tech_data['Price']['SMA_20'],
        'SMA 50': tech_data['Price']['SMA_50'],
        'SMA 200': tech_data['Price']['SMA_200']
    }
    
    fig.add_trace(
        go.Bar(
            x=list(price_data.keys()),
            y=list(price_data.values()),
            name='Price Levels',
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # 2. Fundamental Metrics
    fund_data = summary['Fundamental']
    metrics = []
    values = []
    
    # Only add metrics that have valid values
    if fund_data['P/E_Ratio'] is not None:
        metrics.append('P/E Ratio')
        values.append(fund_data['P/E_Ratio'])
    if fund_data['P/B_Ratio'] is not None:
        metrics.append('P/B Ratio')
        values.append(fund_data['P/B_Ratio'])
    if fund_data['ROE'] is not None:
        metrics.append('ROE')
        values.append(fund_data['ROE'])
    if fund_data['Dividend_Yield'] is not None:
        metrics.append('Dividend Yield')
        values.append(fund_data['Dividend_Yield'])
    
    if metrics:  # Only add if we have valid metrics
        fig.add_trace(
            go.Bar(
                x=metrics,
                y=values,
                name='Fundamental Metrics',
                marker_color='lightgreen'
            ),
            row=1, col=2
        )
    
    # 3. Performance Metrics
    perf_data = summary['Performance']
    returns = {
        '1M': perf_data['1M_Return'],
        '3M': perf_data['3M_Return'],
        '6M': perf_data['6M_Return'],
        '1Y': perf_data['1Y_Return'],
        'YTD': perf_data['YTD_Return']
    }
    
    # Filter out None values
    returns = {k: v for k, v in returns.items() if v is not None}
    
    if returns:  # Only add if we have valid returns
        fig.add_trace(
            go.Bar(
                x=list(returns.keys()),
                y=list(returns.values()),
                name='Returns',
                marker_color='lightcoral'
            ),
            row=2, col=1
        )
    
    # 4. Market Sentiment
    sent_data = summary['Market_Sentiment']
    
    if sent_data['Current_Sentiment'] is not None:
        # Create a simple bar chart instead of gauge
        sentiment_value = sent_data['Current_Sentiment'] * 100
        fig.add_trace(
            go.Bar(
                x=['Market Sentiment'],
                y=[sentiment_value],
                name='Sentiment',
                marker_color='lightseagreen',
                text=[f'{sentiment_value:.1f}%'],
                textposition='auto'
            ),
            row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        title=f'{ticker} Market Summary',
        height=800,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        # Add y-axis labels
        yaxis_title="Price",
        yaxis2_title="Value",
        yaxis3_title="Return (%)",
        yaxis4_title="Sentiment Score"
    )
    
    # Update y-axis ranges for better visualization
    if price_data.values():
        max_price = max(price_data.values())
        fig.update_yaxes(range=[0, max_price * 1.1], row=1, col=1)
    
    if values:
        max_value = max(values)
        fig.update_yaxes(range=[0, max_value * 1.1], row=1, col=2)
    
    if returns:
        min_return = min(returns.values())
        max_return = max(returns.values())
        fig.update_yaxes(range=[min_return * 1.1, max_return * 1.1], row=2, col=1)
    
    if sent_data['Current_Sentiment'] is not None:
        fig.update_yaxes(range=[0, 100], row=2, col=2)
    
    return fig

# Create market summary for each stock
for ticker in analyzers:
    print(f"\nGenerating market summary for {ticker}...")
    try:
        fig = create_market_summary(ticker)
        fig.show()
    except Exception as e:
        print(f"Error creating market summary for {ticker}: {str(e)}")


Generating market summary for AAPL...
Error creating market summary for AAPL: '1M_Return'

Generating market summary for AMZN...
Error creating market summary for AMZN: '1M_Return'

Generating market summary for GOOG...
Error creating market summary for GOOG: '1M_Return'

Generating market summary for META...
Error creating market summary for META: '1M_Return'

Generating market summary for MSFT...
Error creating market summary for MSFT: '1M_Return'

Generating market summary for NVDA...
Error creating market summary for NVDA: '1M_Return'

Generating market summary for TSLA...
Error creating market summary for TSLA: '1M_Return'


## 4. Comparative Analysis

In [5]:
def create_comparative_analysis():
    """Create a comparative analysis dashboard across all stocks."""
    # Prepare data for comparison
    comparison_data = {}
    for ticker in analyzers:
        analyzer = analyzers[ticker]
        summary = analyzer.get_technical_summary()
        comparison_data[ticker] = {
            'RSI': summary['Momentum']['RSI'],
            'MACD_Signal': 1 if summary['Momentum']['MACD_Signal'] == 'Bullish' else -1,
            'BB_Width': summary['Volatility']['BB_Width'],
            'Volatility': summary['Volatility']['Volatility_20d'],
            'MFI': summary['Volume']['MFI']
        }
    
    # Convert to DataFrame
    df_compare = pd.DataFrame(comparison_data).T
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=df_compare.values,
        x=df_compare.columns,
        y=df_compare.index,
        colorscale='RdYlGn',
        zmid=0
    ))
    
    fig.update_layout(
        title='Comparative Technical Analysis',
        xaxis_title='Indicators',
        yaxis_title='Stocks',
        height=600
    )
    
    return fig

# Create comparative analysis
print("\nGenerating comparative analysis...")
fig = create_comparative_analysis()
fig.show()


Generating comparative analysis...
